# 规划问题
## Scipy
### 例题1
<img src="../images/iShot2021-08-09 16.51.12.png"/>

In [1]:
import numpy as np
from scipy import optimize
import pulp
import sys
import math

In [2]:
c = np.array([2, 3, -5])
A = np.array([[-2, 5, -1], [1, 3, 1]])
B = np.array([-10, 12])
Aeq = np.array([[1, 1, 1]])
Beq = np.array([7])

res = optimize.linprog(-c, A, B, Aeq, Beq)  #Max第一个参数为负数，Min为正数
res

     con: array([1.80714554e-09])
     fun: -14.571428565645032
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24602559e-10,  3.85714286e+00])
  status: 0
 success: True
       x: array([6.42857143e+00, 5.71428571e-01, 2.35900788e-10])

x为x1 x2 x3的取值，fun为规划最大值

## pulp
### 例2
<img src="../images/iShot2021-08-09 16.53.29.png"/>

In [3]:
z = [2, 3, 1]

a = [[1, 4, 2], [3, 2, 0]]
b = [8, 6]

c = [[1, 2, 4]]
d = [101]

m = pulp.LpProblem(sense=pulp.LpMaximize)
# x = [pulp.LpVariable(f'x1'), pulp.LpVariable(f'x2'), pulp.LpVariable(f'x3')]
x = [pulp.LpVariable(f'x{i}', lowBound=0, cat=pulp.LpInteger) for i in [1, 2, 3]]
print(x)

m += pulp.lpDot(z, x)
for i in range(len(a)):
    m += (pulp.lpDot(a[i], x) >= b[i])
for i in range(len(c)):
    m += (pulp.lpDot(c[i]
                     , x) == d[i])
for i in x:
    m += (pulp.lpDot([1], i) >= [0])

m.solve()
print(f'y:{pulp.value(m.objective)}')
print(f'x:{[pulp.value(var) for var in x]}')

[x1, x2, x3]
y:202.0
x:[101.0, 0.0, 0.0]


## 整数规划
<img src="../images/iShot2021-08-11 15.42.52.png"/>
sense：LpMaximize（最大优化）/LpMinimize（最小优化）
cat：默认为连续变量，LpInteger为离散变量，LpBinary为二值变量

In [4]:
# Create a new model
m = pulp.LpProblem(sense=pulp.LpMaximize)

# Create variables
x = pulp.LpVariable(cat=pulp.LpBinary, name="x")
y = pulp.LpVariable(cat=pulp.LpBinary, name="y")
z = pulp.LpVariable(cat=pulp.LpBinary, name="z")

# Add constraint: x + 2 y + 3 z <= 4
m += x + 2 * y + 3 * z <= 4, 'c0'
# m += pulp.LpConstraint(e=(x + 2 * y + 3 * z), sense=pulp.LpConstraintLE, rhs=4, name='c0')

# Add constraint: x + y >= 1
m += x + y >= 1, 'c1'
# m += pulp.LpConstraint(e=(x + y), sense=pulp.LpConstraintGE, rhs=1, name='c1')

# Set objective
m.setObjective(x + y + 2 * z)
# m += x + y + 2 * z, 'Obj'

# Calculate with the default CBC optimizer
status = m.solve()

if pulp.LpStatus[status] == 'Optimal':

    for v in m.variables():
        print('%s %g' % (v.name, v.varValue))

    print('Obj: %g' % m.objective.value())

x 1
y 0
z 1
Obj: 3


## 非线性规划

计算1/x+x最小值

In [5]:
def fun(args):
    a = args
    v = lambda x: a / x[0] + x[0]
    return v

In [6]:
args = (1)
x0 = np.asarray((2))  # 猜测初始值
res = optimize.minimize(fun(args), x0, method='SLSQP')
print(res.fun)
print(res.success)
print(res.x)

2.0000000815356342
True
[1.00028559]


<img src="../images/iShot2021-08-11 15.42.53.png"/>

In [7]:
def fun(args):
    a, b, c, d = args
    v = lambda x: (a + x[0]) / (b + x[1]) - c * x[0] + d * x[2]
    return v


def con(args):
    # 约束条件 分为eq 和ineq
    # eq表示 函数结果等于0 ； ineq 表示 表达式大于等于0
    x1min, x1max, x2min, x2max, x3min, x3max = args
    cons = ({'type': 'ineq', 'fun': lambda x: x[0] - x1min},
            {'type': 'ineq', 'fun': lambda x: -x[0] + x1max},
            {'type': 'ineq', 'fun': lambda x: x[1] - x2min},
            {'type': 'ineq', 'fun': lambda x: -x[1] + x2max},
            {'type': 'ineq', 'fun': lambda x: x[2] - x3min},
            {'type': 'ineq', 'fun': lambda x: -x[2] + x3max})
    return cons

In [9]:
#定义常量值
args = (2, 1, 3, 4)  #a,b,c,d
#设置参数范围/约束条件
args1 = (0.1, 0.9, 0.1, 0.9, 0.1, 0.9)  #x1min, x1max, x2min, x2max
cons = con(args1)  #设置初始猜测值
x0 = np.asarray((0.5, 0.5, 0.5))    # 初试权重
res = optimize.minimize(fun(args), x0, method='SLSQP', constraints=cons)
print(res.fun)
print(res.success)
print(res.x)


-0.773684210526435
True
[0.9 0.9 0.1]
